In [ ]:
from pprint import pprint
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

llm_name = "gpt-4"
os.environ["OPENAI_API_KEY"]  # you should see the api key if the .env file is loaded correctly

In [8]:
persist_directory = "/Users/dardonacci/Documents/work/ai-assistant/vector_stores"
embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Prompts

In [9]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [10]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [11]:
query = """Can you explain me how to use the pure market making strategy?
"""

In [12]:
result = qa_chain({"question": query})

In [23]:
result["result"]

"Sure, here's a step-by-step guide on how to use the pure market making strategy with Hummingbot:\n\n1. First, you need to initialize Hummingbot. You can do this by installing the Hummingbot software and running it on your machine.\n\n2. Once Hummingbot is running, you will be prompted to enter your exchange information. This includes the name of the exchange you are using and the trading pair you wish to trade.\n\n3. Next, you will be asked to configure the parameters for the pure market making strategy. These parameters include:\n\n   - `bid_spread`: This is how far away from the mid price you want to place the first bid order.\n   - `ask_spread`: This is how far away from the mid price you want to place the first ask order.\n   - `minimum_spread`: This is the minimum spread at which the bot will automatically cancel orders.\n   - `order_refresh_time`: This is how often you want to cancel and replace bids and asks (in seconds).\n   - `max_order_age`: This is how often you want to can

In [15]:
len(result["source_documents"])

4

# Testing ConverstionalQARetrieval

In [17]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    memory=memory
)

In [19]:
result_conv = qa({"question": query})

In [24]:
result_conv["answer"]

"The Pure Market Making strategy in Hummingbot allows users to run a market making strategy on a single trading pair on spot exchanges. Here's a step-by-step guide on how to use it:\n\n1. Choose your exchange: The first parameter you need to set is the exchange where you want to run your market making strategy. Enter the name of your maker spot connector.\n\n2. Select the trading pair: Enter the token trading pair you would like to trade on the chosen exchange.\n\n3. Set the bid and ask spread: Decide how far away from the mid price you want to place the first bid and ask orders. This is represented as a percentage.\n\n4. Set the minimum spread: This is the minimum spread at which the bot will automatically cancel orders.\n\n5. Set the order refresh time: This is the frequency at which you want to cancel and replace bids and asks. It is measured in seconds.\n\n6. Set the max order age: This is the frequency at which you want to cancel and replace bids and asks with the same price. It i

In [25]:
result_conv_2 = qa.invoke({"question": "How the order refresh time is going to affect my strategy?"})

In [26]:
result_conv_2["answer"]

"The order refresh time is a parameter in the pure market making strategy that determines how often your limit orders are refreshed. For each limit order that was emitted by the pure market making strategy, an expiry timestamp would be generated for that order and the order will be tracked by the strategy. The time until expiry for new orders is configured via the order_refresh_time parameter. After an order's expiration time is reached, the pure market making strategy will create a cancel order proposal for that order. This means that your orders are periodically placed and cancelled according to the proposals made by the order pricing and sizing plugins. This process repeats over and over at each tick, causing limit orders to be periodically placed and cancelled."

In [27]:
result_conv_2 = qa.invoke({"question": "My name is Federico Cardoso"})

In [28]:
result_conv_2["answer"]

"The text doesn't provide the name of a person."

In [29]:
result_conv_2 = qa.invoke({"question": "what's my name?"})

In [30]:
result_conv_2["answer"]

'The text provided does not give any information about the name of an individual in the conversation.'